In [1]:
# Copyright (c) 2019-present, Facebook, Inc.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
#

In [2]:
#
# Code to generate sentence representations from a pretrained model.
# This can be used to initialize a cross-lingual classifier, for instance.
#

In [1]:
import os
import torch

from src.utils import AttrDict
from src.data.dictionary import Dictionary, BOS_WORD, EOS_WORD, PAD_WORD, UNK_WORD, MASK_WORD
from src.model.transformer import TransformerModel

FAISS library was not found.
FAISS not available. Switching to standard nearest neighbors search implementation.


## Reload a pretrained model

download pre-trained model from [this](https://github.com/facebookresearch/XLM#pretrained-cross-lingual-language-models) link

in this notebook we will be work with model `mlm_17_1280.pt`

In [19]:
# create folder for model
os.makedirs('models', exist_ok=True)

# download our pre-trained model
os.system('wget -P ./models https://dl.fbaipublicfiles.com/XLM/mlm_17_1280.pth')

In [21]:
!ls models/

mlm_17_1280.pth


In [22]:
model_path = 'models/mlm_17_1280.pth'
reloaded = torch.load(model_path)
params = AttrDict(reloaded['params'])
print("Supported languages: %s" % ", ".join(params.lang2id.keys()))

Supported languages: ar, de, en, es, fr, hi, it, ja, ko, nl, pl, pt, ru, sv, tr, vi, zh


## Build dictionary / update parameters / build model

In [23]:
# build dictionary / update parameters
dico = Dictionary(reloaded['dico_id2word'], reloaded['dico_word2id'], reloaded['dico_counts'])
params.n_words = len(dico)
params.bos_index = dico.index(BOS_WORD)
params.eos_index = dico.index(EOS_WORD)
params.pad_index = dico.index(PAD_WORD)
params.unk_index = dico.index(UNK_WORD)
params.mask_index = dico.index(MASK_WORD)

# build model / reload weights
model = TransformerModel(params, dico, True, True)
model.eval()
model.load_state_dict(reloaded['model'])

<All keys matched successfully>


## Get sentence representations

Sentences have to be in the BPE format, i.e. tokenized sentences on which you applied fastBPE.

Below you can see an example for English, French, Spanish, German, Arabic and Chinese sentences.

to run to bpe-ize sentences we should:
*  download BPE codes for our downloaded models (the link we get from table where take for model)

In [24]:
# download codes
os.system('wget -P ./models https://dl.fbaipublicfiles.com/XLM/codes_xnli_17')

0

* clone [fastBPE](https://github.com/glample/fastBPE) to folder `tools`

and install it like: `$ g++ -std=c++11 -pthread -O3 fastBPE/main.cc -IfastBPE -o fast` in your work folder

In [30]:
# Below is one way to bpe-ize sentences
codes = './models/codes_xnli_17'
fastbpe = os.path.join(os.getcwd(), 'tools/fastBPE/fast')

In [31]:
# create folder for tmp files
os.makedirs('tmp', exist_ok=True)

In [38]:
def to_bpe(sentences):
    # write sentences to tmp file
    with open('./tmp/sentences', 'w') as fwrite:
        for sent in sentences:
            fwrite.write(sent + '\n')
    
    # apply bpe to tmp file
    os.system('%s applybpe ./tmp/sentences.bpe ./tmp/sentences %s' % (fastbpe, codes))
    
    # load bpe-ized sentences
    sentences_bpe = []
    with open('./tmp/sentences.bpe') as f:
        for line in f:
            sentences_bpe.append(line.rstrip())
    
    return sentences_bpe


In [39]:
# Below are already BPE-ized sentences

# list of (sentences, lang)
sentences = [
    'once he had worn trendy italian leather shoes and jeans from paris that had cost three hundred euros .', # en
    'Le français est la seule langue étrangère proposée dans le système éducatif .', # fr
    'El cadmio produce efectos tóxicos en los organismos vivos , aun en concentraciones muy pequeñas .', # es
    'Nach dem Zweiten Weltkrieg verbreitete sich Bonsai als Hobby in der ganzen Welt .', # de
    'وقد فاز في الانتخابات في الجولة الثانية من التصويت من قبل سيدي ولد الشيخ عبد الله ، مع أحمد ولد داداه في المرتبة الثانية .', # ar
    '羅伯特 · 皮爾 斯 生於 1863年 , 在 英國 曼徹斯特 學習 而 成為 一 位 工程師 . 1933年 , 皮爾斯 在 直布羅陀去世 .', # zh
]

# bpe-ize sentences
sentences = to_bpe(sentences)
print('\n\n'.join(sentences))

# check how many tokens are OOV
n_w = len([w for w in ' '.join(sentences).split()])
n_oov = len([w for w in ' '.join(sentences).split() if w not in dico.word2id])
print('Number of out-of-vocab words: %s/%s' % (n_oov, n_w))

# add </s> sentence delimiters
sentences = [(('</s> %s </s>' % sent.strip()).split()) for sent in sentences]

once he had worn tr@@ endy itali@@ an leather shoes and jeans from paris that had cost three hundred euros .

Le français est la seule langue étrangère proposée dans le système éduc@@ atif .

El cad@@ mio produce efectos tóx@@ icos en los organismos vivos , aun en concentraciones muy pequeñas .

Nach dem Zweiten Weltkrieg verbreitete sich Bon@@ sai als Hob@@ by in der ganzen Welt .

وقد فاز في الانتخابات في الجولة الثانية من التصويت من قبل سيدي ولد الشيخ عبد الله ، مع أحمد ولد دا@@ داه في المرتبة الثانية .

羅@@ 伯特 · 皮@@ 爾 斯 生於 186@@ 3年 , 在 英國 曼@@ 徹@@ 斯特 學習 而 成為 一 位 工程@@ 師 . 1933年 , 皮@@ 爾斯 在 直@@ 布@@ 羅@@ 陀@@ 去世 .
Number of out-of-vocab words: 0/129


In [40]:
!ls tmp/

sentences  sentences.bpe


### Create batch

In [41]:
bs = len(sentences)
slen = max([len(sent) for sent in sentences])

word_ids = torch.LongTensor(slen, bs).fill_(params.pad_index)
for i in range(len(sentences)):
    sent = torch.LongTensor([dico.index(w) for w in sentences[i]])
    word_ids[:len(sent), i] = sent

lengths = torch.LongTensor([len(sent) for sent in sentences])
                             
# NOTE: No more language id (removed it in a later version)
# langs = torch.LongTensor([params.lang2id[lang] for _, lang in sentences]).unsqueeze(0).expand(slen, bs) if params.n_langs > 1 else None
langs = None


### Forward

In [42]:
tensor = model('fwd', x=word_ids, lengths=lengths, langs=langs, causal=False).contiguous()
print(tensor.size())

torch.Size([36, 6, 1280])


The variable `tensor` is of shape `(sequence_length, batch_size, model_dimension)`.

`tensor[0]` is a tensor of shape `(batch_size, model_dimension)` that corresponds to the first hidden state of the last layer of each sentence.

This is this vector that we use to finetune on the GLUE and XNLI tasks.